In [1]:
import sys
import importlib
ARBOR_LOCATION = '/specialhome/lennart/Repos/arbor-sim/arbor/build/python/arbor/__init__.py'
spec = importlib.util.spec_from_file_location('arbor', ARBOR_LOCATION)
module = importlib.util.module_from_spec(spec)
sys.modules[spec.name] = module
spec.loader.exec_module(module)
import iopublic
iopublic.ARBOR_BUILD_CATALOGUE = '/specialhome/lennart/Repos/arbor-sim/arbor/build/arbor-build-catalogue'

In [3]:
import json
import iopublic
import numpy as np
import matplotlib.pyplot as plt
import gzip
sys.path.append('../io/ArborNetworks')
import lib_single
import arbor
import copy
import h5py
import random

%matplotlib inline

def get_tuning(selected):
    fn_tuned = f'tuned_networks/{selected}'
    tuning = json.load(open(fn_tuned))
    for mod in tuning['mods']:
        cal = mod.pop('scal_cal', None) # oops misnamed this one
        if cal is not None:
            mod['cal'] = cal
    return tuning

def run(network, tuning, connected=True):
    recipe = iopublic.TunedIOModel(network, tuning, spikes=())
    if not connected:
        recipe.ggap = 0
    context = arbor.context(threads=8, gpu_id=0)
    domains = arbor.partition_load_balance(recipe, context)
    sim = arbor.simulation(recipe, domains, context)
    handles = [sim.sample((gid, 0), arbor.regular_schedule(1)) for gid in range(recipe.num_cells())]
    sim.run(tfinal=8000, dt=0.025)
    traces = [sim.samples(handle)[0][0].T for handle in handles]
    vsall = np.array([vs for t, vs in traces])
    return vsall[:, -3000:]

def impulse_response(tuning):
    tuning = copy.deepcopy(tuning)
    for mod in tuning['mods']:
        mod.clear()
        mod.update(dict(
                scal_na_s=0, scal_kdr=0, scal_ks=0, scal_cal=0, scal_cah=0,
                scal_kca=0, scal_h=0, scal_cacc=0, scal_na_a=0, scal_k=0,
                Vm=0, Vdend=10, cal=0))
    recipe = iopublic.TunedIOModel(network, tuning, spikes=())
    recipe.ggap = 0
    context = arbor.context(threads=128)
    domains = arbor.partition_load_balance(recipe, context)
    sim = arbor.simulation(recipe, domains, context)
    handles = [sim.sample((gid, 0), arbor.regular_schedule(0.01)) for gid in range(recipe.num_cells())]
    sim.run(tfinal=10, dt=0.01/2)
    traces = [sim.samples(handle)[0][0].T for handle in handles]
    vs = np.array([vs for t, vs in traces])
    x1 = (vs[:,10]-vs[:,0])/10
    x2 = 10-vs[:,-1]
    return vs, x1, x2

# from sim_titan.py in llandsmeer/io
random.seed(1234) # constant pool of randoms
pick_samples = [random.randint(0,999) for _i in range(10000)]
names='scal_ks scal_cal scal_cah scal_kca scal_h scal_cacc'.split()
with gzip.open('../io/ArborNetworks/mech_config_version7.json.gz', 'rt') as f:
    mech_config = json.load(f)
def get_sc_mod(nid, x1, x2, mech_config):
    b1, b2 = x1[nid]*100-.5, x2[nid]*100-.5
    b = min(mech_config, key=lambda ex: (b1 - ex['bucket'][0])**2 + (b2 - ex['bucket'][1])**2)
    opt = b['opt'][pick_samples[nid] % len(b['opt'])] # consistant same sample across sims
    config = list(opt['ind'])
    mod = dict(zip(names, config))
    return mod


In [59]:
%%time

selected = '2021-12-08-shadow_averages_0.001_0.8_d1666304-c6fc-4346-a55d-a99b3aad55be'

network = iopublic.get_network_for_tuning(selected)

C = np.array([neuron.cluster for neuron in network.neurons])

tuning = get_tuning(selected)
%time vsall_conn_tune = run(network, tuning, connected=True)
%time vsall_unconn_tune = run(network, tuning, connected=False)

for mod in tuning['mods']:
    mod.clear()

%time vsall_conn_none = run(network, tuning, connected=True)
%time vsall_unconn_none = run(network, tuning, connected=False)

vstun, x1, x2 = impulse_response(tuning)
for gid, mod in enumerate(tuning['mods']):
    mod.clear()
    mod.update(get_sc_mod(gid, x1, x2, mech_config))

%time vsall_conn_sc = run(network, tuning, connected=True)
%time vsall_unconn_sc = run(network, tuning, connected=False)

I = 20 
vstun, x1, x2 = impulse_response(tuning)
for gid, mod in enumerate(tuning['mods']):
    mod.clear()
    mod.update(get_sc_mod(I, x1, x2, mech_config))
    
%time vsall20 = run(network, tuning, connected=False)


import h5py
with h5py.File('./imgdata/control_tuning_v2.h5', 'a') as f:
    f.create_dataset(f'{selected}/tune/conn', data=vsall_conn_tune)
    f.create_dataset(f'{selected}/tune/unconn', data=vsall_unconn_tune)
    f.create_dataset(f'{selected}/sc/conn', data=vsall_conn_sc)
    f.create_dataset(f'{selected}/sc/unconn', data=vsall_unconn_sc)
    f.create_dataset(f'{selected}/none/conn', data=vsall_conn_none)
    f.create_dataset(f'{selected}/none/unconn', data=vsall_unconn_none)
    f.create_dataset(f'{selected}/gid20/unconn', data=vsall20)
    f.create_dataset(f'{selected}/x1', data=x1)
    f.create_dataset(f'{selected}/x2', data=x2)
    f.create_dataset(f'{selected}/vstune', data=vstun)
    f.create_dataset(f'{selected}/C', data=C)

CPU times: user 3min 29s, sys: 4min 16s, total: 7min 45s
Wall time: 7min 42s
CPU times: user 11min 4s, sys: 4min 22s, total: 15min 26s
Wall time: 7min 44s
CPU times: user 10min 31s, sys: 4min 7s, total: 14min 39s
Wall time: 7min 20s
CPU times: user 3min 18s, sys: 4min 5s, total: 7min 23s
Wall time: 7min 20s
CPU times: user 11min 15s, sys: 4min 24s, total: 15min 40s
Wall time: 7min 51s
CPU times: user 11min 19s, sys: 4min 29s, total: 15min 48s
Wall time: 7min 55s
CPU times: user 54min 14s, sys: 25min 45s, total: 1h 20min
Wall time: 47min 34s
